In [1]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle
import glob

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var
from object_pose_utils.utils.display import *

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


## Dataset Information

In [2]:
dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'

non_sym_objs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 17, 18,]
sym_objs = [13, 16, 19, 20, 21]
textureless_objects = set([10, 11, 13, 14, 16, 19, 20, 21])
textured_objects = set(range(1,22)) - set([10, 11, 13, 14, 16, 19, 20, 21])


with open('{0}/image_sets/classes.txt'.format(dataset_root)) as f:                                    
    classes = f.read().split()
classes.insert(0, '__background__')

In [3]:
def recursiveUpdate(dict_base, dict_new):
    for k,v in dict_new.items():
        if(k in dict_base):
            dict_base[k].update(v)
        else:
            dict_base[k] = v

def updateResults(results_dict, new_results_filename):
    new_data = np.load(new_results_filename, allow_pickle=True)
    new_results = {k:v.item() for k,v in new_data.items()}
    recursiveUpdate(results_dict, new_results)

In [4]:
from object_pose_utils.utils.pose_error import accuracyAUC

def non_func(x):
    return x

def mean_abs(x):
    return np.mean(np.abs(x))

def log_clean(x):
    return np.log(np.maximum(1e-6, x))

def mean_log_clean(x):
    return np.mean(log_clean(x))

def mean_log(x):
    return np.mean(np.log(x))   

def accuracyAUC100(x, max_theshold):
    return accuracyAUC(x, max_theshold)*100

def median_log(x):
    return np.median(np.log(x))

def makeTableEntries(data_dict, keys = None, objs = list(range(1,22)), 
              val_func = non_func, bold_func = max,
              individual = False,
              agg_title = 'All'):
    if(keys is None):
        keys = list(data_dict.keys())
    
    data_str = ''
    if(individual):
        for obj in objs:    
            data_str += ' '.join(classes[obj].split('_')[1:]) + '\n'
            vals = [val_func(np.array(data_dict[k][obj])) for k in keys]
            bold_val = bold_func(vals)
            for v in vals:
                if(v == bold_val):
                    data_str += ' & \\textbf{' + '{:0.2f}'.format(v) + '}'
                else:
                    data_str += ' & {:0.2f}'.format(v)
            data_str += ' \\\\'
            data_str += '\n'

        data_str += '\hline\n'

    data_str += '{} \n'.format(agg_title)
    vals = [val_func(np.concatenate([np.array(data_dict[k][o]) for o in objs])) for k in keys]
    bold_val = bold_func(vals)
    for v in vals:
        if(v == bold_val):
            data_str += ' & \\textbf{' + '{:0.2f}'.format(v) + '}'
        else:
            data_str += ' & {:0.2f}'.format(v)
    data_str += ' \\\\'
    data_str += '\n'
    #print(data_str)
    return data_str
    
def makeTableHeading(names, heading = None):
    data_str = ''
    data_str +='\\begin{table*}[h!]\n'
    data_str +='\\small\n'
    data_str +='\\centering\n'
    data_str +='\\begin{tabular} {||c||' + len(names)*'c|' +'|}\n'
    data_str +='\\hline\n'
        
    data_str += ' '
    if(heading is not None):
        data_str += heading
    else:
        for k in names:
            data_str += ' & ' + k
        data_str += '\\\\'
    data_str += '\n'

    data_str += '\hline\n'
    
    #print(data_str)
    return data_str

def makeTableEnding(caption = None, label = None):
    data_str = ''
    data_str +='\\hline\n'
    data_str +='\\end{tabular}\n'
    if(caption is not None):
        data_str +='\\caption{' + caption + '}\n'
    if(label is not None):
        data_str +='\\label{tbl:' + label + '}\n'
    data_str +='\\end{table*}\n'
    #print(data_str)
    return data_str

In [298]:
results_keys = ['likelihood', 'sym_angular_error_mode', 'add_error_mode', 'add_sym_error_mode']

###################
### DenseFusion ###
###################

df_algorithm_keys = ['bing_iso',
                     #'bing_duel',
                     'hist_comp_global', 
                     'hist_uniform',
                     'bing_fixed',
                     'bing_dropout',
                     'hist_conf', 
                     'hist_cosine', 
                     #'bing_mixed',
                     'hist_reg_global', 
                    ]

df_results = {}

for fn in glob.glob('ral_results/indiv_new/df_*'):
    updateResults(df_results, fn)
    
for res_k, res_v in df_results.items():
    for alg_k, alg_v in res_v.items():
        for k, v in alg_v.items():
            if(type(v) is dict):
                df_results[res_k][alg_k][k] = list(v.values())

for res_k in df_results.keys():
    for alg_k in df_algorithm_keys:
        if(alg_k not in df_results[res_k].keys()):
            df_results[res_k][alg_k] = {}
            for k in range(1,22):
                df_results[res_k][alg_k][k] = [np.nan]
                
###############                
### PoseCNN ###
###############

pc_algorithm_keys = ['bing_iso',
                     #'bing_duel',
                     'hist_comp_fc6', 
                     'hist_uniform',
                     'bing_fixed',
                     'bing_dropout',
                     'hist_conf', 
                     'hist_cosine', 
                     #'bing_mixed',
                     'hist_reg_fc6', 
                     ]

pc_results = {}

for fn in glob.glob('ral_results/indiv_new/pcnn_*'):
    updateResults(pc_results, fn)
                
for res_k, res_v in pc_results.items():
    for alg_k, alg_v in res_v.items():
        for k, v in alg_v.items():
            if(type(v) is dict):
                pc_results[res_k][alg_k][k] = list(v.values())

for res_k in pc_results.keys():
    for alg_k in pc_algorithm_keys:
        if(alg_k not in pc_results[res_k].keys()):
            pc_results[res_k][alg_k] = {}
            for k in range(1,22):
                pc_results[res_k][alg_k][k] = [np.nan]

In [232]:
for fn in glob.glob('ral_results/*_results.npz', recursive=True):
    try:
        data = np.load(fn, allow_pickle=True)
        test_results = {'likelihood':data['likelihood'].item()}
        #updateResults(test_results, fn)

        #print(test_results['likelihood'].keys())
        to_list(test_results)
        k = 'hist_cosine'
        if(k in test_results['likelihood'].keys()):
            print(fn)
            print(makeTableEntries(test_results['likelihood'], keys=[k], objs = non_sym_objs, 
                 val_func=mean_log_clean, bold_func = max))
    except:
        print('x'*50)
        print(fn)
        print('x'*50)

ral_results/single_img_hist_cosine_results.npz
All 
 & \textbf{-1.90} \\

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
ral_results/single_img_hist_reg_local_hist_reg_global_hist_comp_local_hist_comp_global_bing_fixed_bing_mixed_results.npz
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


Exception ignored in: <bound method NpzFile.__del__ of <numpy.lib.npyio.NpzFile object at 0x7fbb39e164e0>>
Traceback (most recent call last):
  File "/home/bokorn/src/generic_pose/bpy/lib/python3.6/site-packages/numpy/lib/npyio.py", line 230, in __del__
    self.close()
  File "/home/bokorn/src/generic_pose/bpy/lib/python3.6/site-packages/numpy/lib/npyio.py", line 221, in close
    if self.zip is not None:
AttributeError: 'NpzFile' object has no attribute 'zip'


ral_results/pcnn_single_img_hist_cosine_results.npz
All 
 & \textbf{-0.92} \\

ral_results/pcnn_single_img_fused_hist_reg_fc6_hist_cosine_hist_uniform_bing_fixed_results.npz
All 
 & nan \\



In [231]:
for fn in glob.glob('ral_results/*_results.npz', recursive=True):
    try:
        data = np.load(fn, allow_pickle=True)
        test_results = {'likelihood':data['likelihood'].item()}
        #updateResults(test_results, fn)

        #print(test_results['likelihood'].keys())
        to_list(test_results)
        k = 'hist_cosign'
        if(k in test_results['likelihood'].keys()):
            print(fn)
            print(makeTableEntries(test_results['likelihood'], keys=[k], objs = non_sym_objs, 
                 val_func=mean_log_clean, bold_func = max))
    except:
        print('x'*50)
        print(fn)
        print('x'*50)

ral_results/single_img_cosin_orig_results.npz
All 
 & \textbf{-2.16} \\

ral_results/pcnn_single_imghist_reg_global_hist_comp_global_hist_cosign_hist_uniform_bing_fixed_results.npz
All 
 & nan \\

ral_results/single_imghist_cosign_results.npz
All 
 & \textbf{-2.16} \\

ral_results/pcnn_single_img1_hist_reg_global_hist_comp_global_hist_cosign_hist_uniform_bing_fixed_results.npz
All 
 & \textbf{-0.89} \\

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
ral_results/single_img_hist_reg_local_hist_reg_global_hist_comp_local_hist_comp_global_bing_fixed_bing_mixed_results.npz
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


Exception ignored in: <bound method NpzFile.__del__ of <numpy.lib.npyio.NpzFile object at 0x7fbb4cc889b0>>
Traceback (most recent call last):
  File "/home/bokorn/src/generic_pose/bpy/lib/python3.6/site-packages/numpy/lib/npyio.py", line 230, in __del__
    self.close()
  File "/home/bokorn/src/generic_pose/bpy/lib/python3.6/site-packages/numpy/lib/npyio.py", line 221, in close
    if self.zip is not None:
AttributeError: 'NpzFile' object has no attribute 'zip'


ral_results/pcnn_single_img2_hist_reg_global_hist_comp_global_hist_cosign_hist_uniform_bing_fixed_results.npz
All 
 & nan \\



In [191]:
ls ral_results/indiv/

df_bing_dropout.npz              df_hist_uniform.npz
df_bing_duel.npz                 pcnn_bing_dropout.npz
df_bing_fixed.npz                pcnn_bing_duel.npz
df_bing_iso.npz                  pcnn_bing_fixed.npz
df_bing_mixed.npz                pcnn_bing_iso.npz
df_hist_comp_global.npz          pcnn_hist_comp_fc6.npz
df_hist_comp_local.npz           pcnn_hist_conf.npz
df_hist_conf.npz                 pcnn_hist_cosine.npz
df_hist_reg_global.npz           pcnn_hist_reg_fc6.npz
df_hist_reg_indv_global_all.npz  pcnn_hist_uniform.npz
df_hist_reg_local.npz


In [239]:
ls ral_results/indiv_new/

df_bing_dropout.npz      df_hist_conf.npz        pcnn_bing_fixed.npz
df_bing_duel.npz         df_hist_reg_global.npz  pcnn_bing_iso.npz
df_bing_fixed.npz        df_hist_reg_local.npz   pcnn_hist_comp_fc6.npz
df_bing_iso.npz          df_hist_uniform.npz     pcnn_hist_conf.npz
df_hist_comp_global.npz  pcnn_bing_dropout.npz   pcnn_hist_reg_fc6.npz
df_hist_comp_local.npz   pcnn_bing_duel.npz      pcnn_hist_uniform.npz


ls: cannot access 'ral_results/indiv_new/z': No such file or directory


In [235]:
fn = 'ral_results/pcnn_single_img_hist_cosine_results.npz'
data = np.load(fn, allow_pickle=True)
list(data['likelihood'].item().keys())

['hist_cosine']

In [241]:
#fn = 'ral_results/final_single_img_hist_reg_local_hist_reg_global_hist_comp_local_hist_comp_global_bing_iso_bing_duel_results.npz'
#fn = 'ral_results/final_single_img_bing_fixed_results.npz'
#fn = 'ral_results/final_pcnn_single_img_bing_iso_bing_duel_results.npz'
#fn = 'ral_results/final_pcnn_single_img_bing_fixed_results.npz'
#fn = 'ral_results/pcnn_single_img_bing_dropout_results.npz'
#fn = 'ral_results/single_img_dropout_50_orig_results.npz'
#fn = 'ral_results/pcnn_single_img_fused_hist_reg_fc6_hist_cosine_hist_uniform_bing_fixed_results.npz'
#fn = 'ral_results/single_img_results.npz'
#fn = 'ral_results/pcnn_single_img_hist_conf_bing_iso_bing_duel_results.npz'
fn = 'ral_results/single_img_hist_cosine_results.npz'
#fn = 'ral_results/pcnn_single_img_hist_cosine_results.npz'

data = np.load(fn, allow_pickle=True)

for alg_k in data['likelihood'].item().keys():
    results = {}
    if(alg_k != 'hist_cosine'):
        continue
    for res_k in data.keys():
        if(res_k == 'lik_distribution'):
            continue
        results[res_k] = {alg_k:data[res_k].item()[alg_k]}
    print(alg_k)
    np.savez('ral_results/indiv_new/df_{}.npz'.format(alg_k),
             **results)

hist_cosine


In [119]:
#data = np.load(fn, allow_pickle=True)
fn = 'ral_results/final_single_img_hist_reg_local_hist_reg_global_hist_comp_local_hist_comp_global_bing_iso_bing_duel_results.npz'
test_results = {}
updateResults(test_results, fn)
to_list(test_results)
print(list(test_results['likelihood'].keys()))
print(makeTableEntries(test_results['likelihood'], objs = non_sym_objs, 
                 val_func=mean_log_clean, bold_func = max))

['hist_reg_local', 'hist_reg_global', 'hist_comp_local', 'hist_comp_global', 'bing_iso', 'bing_duel']
All 
 & 0.45 & 0.28 & 0.73 & 1.18 & \textbf{2.80} & 2.56 \\



In [59]:
def to_list(results):
    for res_k, res_v in results.items():
        for alg_k, alg_v in res_v.items():
            for k, v in alg_v.items():
                if(res_k not in results.keys()):
                    test_results[res_k] = {}
                if(alg_k not in results[res_k].keys()):
                    test_results[res_k][alg_k] = {}
                if(k not in results[res_k][alg_k].keys()):
                    test_results[res_k][alg_k][k] = {}
                if(type(v) is dict):
                    results[res_k][alg_k][k] = list(v.values())


In [64]:
#for fn in glob.glob('ral_results/*cos*'):
#    if('pcnn' not in fn):
#        print(fn)
test_results1 = {}
updateResults(test_results1, 'ral_results/single_img_hist_cosine_results.npz')
test_results2 = {}
updateResults(test_results2, 'ral_results/single_imghist_cosign_results.npz')
test_results3 = {}
updateResults(test_results3, 'ral_results/single_img_cosin_orig_results.npz')

print(test_results1['likelihood'].keys())
print(test_results2['likelihood'].keys())
print(test_results3['likelihood'].keys())

to_list(test_results1)
to_list(test_results2)
to_list(test_results3)

print('='*50)

print(makeTableEntries(df_results['likelihood'], keys=['hist_cosine'], objs = non_sym_objs, 
                 val_func=mean_log_clean, bold_func = max,
                 agg_title = 'Non-Symmetric-DF'))
print(makeTableEntries(pc_results['likelihood'], keys=['hist_cosine'], objs = non_sym_objs, 
                 val_func=mean_log_clean, bold_func = max,
                 agg_title = 'Non-Symmetric-DF'))
print('-'*50)
print(makeTableEntries(test_results1['likelihood'], keys=['hist_cosine'], objs = non_sym_objs, 
                 val_func=mean_log_clean, bold_func = max,
                 agg_title = 'Non-Symmetric-DF'))
print(makeTableEntries(test_results2['likelihood'], keys=['hist_cosign'], objs = non_sym_objs, 
                 val_func=mean_log_clean, bold_func = max,
                 agg_title = 'Non-Symmetric-DF'))
print(makeTableEntries(test_results3['likelihood'], keys=['hist_cosign'], objs = non_sym_objs, 
                 val_func=mean_log_clean, bold_func = max,
                 agg_title = 'Non-Symmetric-DF'))


dict_keys(['hist_cosine'])
dict_keys(['hist_cosign'])
dict_keys(['hist_cosign'])
Non-Symmetric-DF 
 & nan \\

Non-Symmetric-DF 
 & \textbf{-1.90} \\

--------------------------------------------------
Non-Symmetric-DF 
 & \textbf{-1.90} \\

Non-Symmetric-DF 
 & \textbf{-2.16} \\

Non-Symmetric-DF 
 & \textbf{-2.16} \\



In [6]:
print(df_results['likelihood']['bing_fixed'][1][0])
print(np.mean(np.log(df_results['likelihood']['bing_fixed'][1][0])))
print(np.load('df_new_sigmas.npz', allow_pickle=True)['single_sigma'].item()[1])
len(df_results['likelihood']['bing_fixed'][1])

45.337440490722656
3.814133192024154
53.06759009009921


1006

In [7]:
hist_global_results = 'ral_results/single_img_hist_orig_results.npz'
hist_conf_results = 'ral_results/single_img_hist_conf_orig_results.npz'
bing_fixed_results = 'ral_results/single_img_bing_results.npz'
hist_cos_results = 'ral_results/single_img_hist_cosine_results.npz'
hist_uniform_results = 'ral_results/single_img_uniform_results.npz'
hist_bing_results = 'ral_results/single_img_bing_iso_bing_duel_results.npz'
hist_dropout_results = 'ral_results/single_img_dropout_50_orig_results.npz'

results_1 = 'ral_results/pcnn_single_img_hist_cosine_results.npz'
results_2 = 'ral_results/pcnn_single_img_fused_hist_reg_fc6_hist_cosine_hist_uniform_bing_fixed_results.npz'
results_3 = 'ral_results/pcnn_single_img_hist_comp_fc6_results.npz'
results_4 = 'ral_results/pcnn_single_img_hist_conf_bing_iso_bing_duel_results.npz'
results_5 = 'ral_results/pcnn_single_img_bing_dropout_results.npz'

results_bing_reg = 'ral_results/final_pcnn_single_img_bing_iso_bing_duel_results.npz'

In [297]:
table_heading_old = """ &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{5}{c||}{Bingham} \\\\\n\\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Mix & Dropout & Reg Iso & Reg Full \\\\"""

""" 
 &  & \\multicolumn{4}{c||}{Histogram} & \\multicolumn{5}{c||}{Bingham} \\\
\\hline
 &  & Stat & \\multicolumn{3}{c||}{Learned} & \\multicolumn{3}{c||} Stat & \\multicolumn{2}{c||} Learned \\\
\\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Mix & Dropout & Reg Iso & Reg Full \\\
""";

table_heading = """ &  & \multicolumn{2}{c||}{Our Method} & \multicolumn{5}{c||}{Baselines} \\\\\n\\hline
Objects & Bingham Regression & Learned Comparison & Uniform & Fixed Bingham & Dropout & Confusion Matrix & Cosine Distance & Histogram Regression \\\\"""



In [302]:
#df_v['hist_cosine']
from object_pose_utils.bbTrans.discretized4dSphere import S3Grid

#grid = S3Grid(2)
#grid.Simplify()
num_verts = len(grid.vertices)
print(num_verts)

3885


In [300]:
k = 'likelihood'
df_v = df_results[k]
pc_v = pc_results[k]

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in df_v.keys()],
                             heading = table_heading)

table_str += makeTableEntries(df_v, keys=df_algorithm_keys, objs = non_sym_objs, 
                              val_func=mean_log_clean, bold_func = max,
                              agg_title = 'Non-Symmetric-DF')
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys, objs = non_sym_objs, 
                              val_func=mean_log_clean, bold_func = max,
                              agg_title = 'Non-Symmetric-PC')
table_str += "\\hline\n"
table_str += "\\hline\n"
table_str += makeTableEntries(df_v, keys=df_algorithm_keys, objs = sym_objs,
                              val_func=mean_log_clean, bold_func = max, 
                              agg_title = 'Symmetric-DF')
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys, objs = sym_objs,
                              val_func=mean_log_clean, bold_func = max, 
                              agg_title = 'Symmetric-PC')
table_str += "\\hline\n"
table_str += "\\hline\n"
table_str += makeTableEntries(df_v, keys=df_algorithm_keys, val_func=mean_log_clean, bold_func = max, 
                             agg_title = 'All-DF')
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys, val_func=mean_log_clean, bold_func = max, 
                             agg_title = 'All-PC')
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'Mean Log Likelihood of Ground Truth Orientation')
print(table_str)

\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{2}{c||}{Our Method} & \multicolumn{5}{c||}{Baselines} \\
\hline
Objects & Bingham Regession & Learned Comparison & Uniform & Fixed Bingham & Dropout & Confusion Matrix & Cosine Distance & Histogram Regression \\
\hline
Non-Symmetric-DF 
 & \textbf{2.80} & 1.18 & -2.29 & 1.74 & 0.70 & 1.63 & -1.90 & 0.28 \\
Non-Symmetric-PC 
 & 1.91 & 2.17 & -2.29 & 1.50 & \textbf{2.71} & -2.46 & -0.92 & 1.87 \\
\hline
\hline
Symmetric-DF 
 & -3.81 & -5.54 & -2.29 & -3.66 & -8.09 & -2.91 & \textbf{-2.23} & -2.57 \\
Symmetric-PC 
 & -8.82 & \textbf{-0.52} & -2.29 & -9.18 & -5.28 & -7.75 & -1.55 & -1.23 \\
\hline
\hline
All-DF 
 & \textbf{1.72} & 0.08 & -2.29 & 0.86 & -0.74 & 0.88 & -1.95 & -0.19 \\
All-PC 
 & 0.19 & \textbf{1.74} & -2.29 & -0.22 & 1.43 & -3.31 & -1.02 & 1.37 \\
\hline
\end{tabular}
\caption{Mean Log Likelihood of Ground Truth Orientation}
\end{table*}



In [21]:
k = 'add_error_mode'
df_v = df_results[k]
pc_v = pc_results[k]

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in df_v.keys()],
                              heading = table_heading)
table_str += makeTableEntries(df_v, keys=df_algorithm_keys,
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                             individual = True)
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'AUC of Average Distance Error of Distribution Mode')
print(table_str)
print("\n\n\n\n")

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in pc_v.keys()],
                              heading = table_heading)
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys,
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                             individual = True)
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'AUC of Average Distance Error of Distribution Mode')
print(table_str)
print("\n\n\n\n")


\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{5}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Mix & Dropout & Reg Iso & Reg Full \\
\hline
master chef can
 & 0.00 & 65.45 & 56.85 & 56.36 & 49.81 & \textbf{68.89} & \textbf{68.89} & 12.75 & 68.87 & 68.87 \\
cracker box
 & 0.00 & 80.56 & 52.27 & 64.08 & 61.75 & 83.78 & 83.78 & 4.40 & \textbf{83.90} & \textbf{83.90} \\
sugar box
 & 5.85 & 87.75 & 80.27 & 80.42 & 76.32 & 91.01 & 91.01 & 10.24 & \textbf{91.04} & \textbf{91.04} \\
tomato soup can
 & 27.78 & 82.05 & 79.41 & 82.80 & 80.37 & 84.48 & 84.48 & 15.47 & \textbf{84.49} & \textbf{84.49} \\
mustard bottle
 & 0.81 & 88.23 & 80.18 & 80.47 & 79.42 & \textbf{91.92} & \textbf{91.92} & 13.79 & 91.76 & 91.76 \\
tuna fish can
 & 50.86 & 77.78 & 72.51 & 74.24 & 70.42 & \textbf{79.94} & \textbf{79.94} & 19.50 & 79.93 & 79.93 \\
pudding box
 & 18.17 & 81.42 & 65.09 & 

In [22]:
k = 'add_sym_error_mode'
df_v = df_results[k]
pc_v = pc_results[k]

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in df_v.keys()],
                              heading = table_heading)
table_str += makeTableEntries(df_v, keys=df_algorithm_keys,
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                             individual = True)
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'AUC of Average Symetric Distance Error of Distribution Mode using DenseFusion Features')
print(table_str)
print("\n\n\n\n")

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in pc_v.keys()],
                              heading = table_heading)
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys,
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                             individual = True)
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'AUC of Average Distance Error of Distribution Mode using PoseCNN Features')
print(table_str)
print("\n\n\n\n")


\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{5}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Mix & Dropout & Reg Iso & Reg Full \\
\hline
master chef can
 & 85.00 & 93.89 & 92.88 & 93.27 & 93.09 & 94.39 & 94.39 & 40.20 & \textbf{94.41} & \textbf{94.41} \\
cracker box
 & 70.90 & 90.66 & 85.45 & 86.78 & 86.97 & 91.73 & 91.73 & 22.71 & \textbf{91.78} & \textbf{91.78} \\
sugar box
 & 77.29 & 94.08 & 91.50 & 91.35 & 89.80 & 95.34 & 95.34 & 31.61 & \textbf{95.39} & \textbf{95.39} \\
tomato soup can
 & 88.35 & 95.15 & 93.92 & 94.95 & 94.77 & \textbf{95.70} & \textbf{95.70} & 33.12 & 95.69 & 95.69 \\
mustard bottle
 & 79.79 & 95.12 & 92.27 & 92.38 & 92.20 & \textbf{96.30} & \textbf{96.30} & 41.88 & 96.26 & 96.26 \\
tuna fish can
 & 90.83 & 95.60 & 94.54 & 95.34 & 93.73 & \textbf{96.00} & \textbf{96.00} & 44.76 & 95.99 & 95.99 \\
pudding box
 & 87.00 & 92.54 & 88.

In [27]:
k = 'sym_angular_error'
df_v = df_results[k]
pc_v = pc_results[k]

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in df_v.keys()],
                              heading = table_heading)
table_str += makeTableEntries(df_v, keys=df_algorithm_keys,
                              val_func=partial(accuracyAUC100, max_theshold=180), bold_func = max, 
                             individual = True)
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'AUC of Symetric Angular Error of Distribution Mode using DenseFusion')
print(table_str)
print("\n\n\n\n")

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in pc_v.keys()],
                              heading = table_heading)
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys,
                              val_func=partial(accuracyAUC100, max_theshold=180), bold_func = max, 
                             individual = True)
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'AUC of Symetric Angular Error of Distribution Mode using PoseCNN')
print(table_str)
print("\n\n\n\n")


\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{5}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Mix & Dropout & Reg Iso & Reg Full \\
\hline
master chef can
 & 70.49 & 70.49 & 70.49 & 70.49 & 70.49 & \textbf{70.53} & \textbf{70.53} & 56.10 & 70.49 & 70.49 \\
cracker box
 & \textbf{94.29} & \textbf{94.29} & \textbf{94.29} & \textbf{94.29} & \textbf{94.29} & 94.19 & 94.19 & 68.33 & \textbf{94.29} & \textbf{94.29} \\
sugar box
 & \textbf{96.39} & \textbf{96.39} & \textbf{96.39} & \textbf{96.39} & \textbf{96.39} & 96.38 & 96.38 & 77.08 & \textbf{96.39} & \textbf{96.39} \\
tomato soup can
 & \textbf{83.92} & \textbf{83.92} & \textbf{83.92} & \textbf{83.92} & \textbf{83.92} & 83.86 & 83.86 & 69.35 & \textbf{83.92} & \textbf{83.92} \\
mustard bottle
 & 93.61 & 93.61 & 93.61 & 93.61 & 93.61 & \textbf{93.77} & \textbf{93.77} & 65.95 & 93.61 & 93.61 \\
tuna fish can
 

In [37]:
#k = 'sym_angular_error'
k = 'likelihood'
df_v = df_results[k]
pc_v = pc_results[k]

print(pc_v['hist_uniform'][1][1])
print(pc_v['hist_reg_fc6'][1][1])

0.10132116079330444
60.02710172876286


In [25]:
k = 'likelihood'
df_v = df_results[k]
pc_v = pc_results[k]

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in df_v.keys()],
                              heading = table_heading)
table_str += makeTableEntries(df_v, keys=df_algorithm_keys,
                              val_func=mean_log_clean, bold_func = max, 
                             individual = True)
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'Mean Log Likelihood of Ground Truth Orientation using DenseFusion')
print(table_str)
print("\n\n\n\n")

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in pc_v.keys()],
                              heading = table_heading)
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys,
                              val_func=mean_log_clean, bold_func = max, 
                             individual = True)
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'Mean Log Likelihood of Ground Truth Orientation using PoseCNN')
print(table_str)
print("\n\n\n\n")


\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{5}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Mix & Dropout & Reg Iso & Reg Full \\
\hline
master chef can
 & -2.29 & -0.78 & -2.82 & 1.01 & -1.91 & -0.97 & -0.94 & -1.04 & -1.66 & \textbf{1.21} \\
cracker box
 & -2.29 & \textbf{4.03} & -1.00 & 2.26 & -1.91 & 1.13 & 1.68 & 1.14 & 3.75 & 1.81 \\
sugar box
 & -2.29 & 3.77 & 2.64 & 2.24 & -1.80 & 0.04 & 0.97 & 1.95 & \textbf{5.94} & -0.06 \\
tomato soup can
 & -2.29 & 0.90 & -2.24 & 1.61 & -1.73 & -0.27 & -0.02 & 1.16 & \textbf{2.02} & 1.63 \\
mustard bottle
 & -2.29 & 3.85 & -1.21 & 2.15 & -1.73 & 2.38 & 2.41 & 1.12 & \textbf{4.61} & 1.73 \\
tuna fish can
 & -2.29 & -3.12 & -5.86 & \textbf{0.42} & -2.05 & -1.17 & -2.19 & -0.96 & -0.20 & 0.31 \\
pudding box
 & -2.29 & 2.18 & -0.18 & -1.35 & -1.95 & 1.15 & 1.19 & 0.82 & \textbf{2.64} & 0.47 \\
gelatin box
 & -2.2

In [23]:
k = 'likelihood'
df_v = df_results[k]
pc_v = pc_results[k]

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in df_v.keys()],
                              heading = table_heading)
table_str += makeTableEntries(df_v, keys=df_algorithm_keys,
                              val_func=mean_log_clean, bold_func = max, 
                             individual = True)
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'AUC of Average Distance Error of Distribution Mode')
print(table_str)
print("\n\n\n\n")

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in pc_v.keys()],
                              heading = table_heading)
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys,
                              val_func=mean_log_clean, bold_func = max, 
                             individual = True)
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'AUC of Average Distance Error of Distribution Mode')
print(table_str)
print("\n\n\n\n")


k = 'add_sym_error_mode'
df_v = df_results[k]
pc_v = pc_results[k]

table_str = ''
table_str += makeTableHeading([h.replace('_', ' ') for h in v_df.keys()],
                              heading = table_heading)

table_str += makeTableEntries(df_v, keys=df_algorithm_keys, objs = non_sym_objs, 
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max,
                              agg_title = 'Non-Symmetric-DF')
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys, objs = non_sym_objs, 
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max,
                              agg_title = 'Non-Symmetric-PC')
table_str += "\\hline\n"
table_str += "\\hline\n"
table_str += makeTableEntries(df_v, keys=df_algorithm_keys, objs = sym_objs,
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                              agg_title = 'Symmetric-DF')
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys, objs = sym_objs,
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                              agg_title = 'Symmetric-PC')
table_str += "\\hline\n"
table_str += "\\hline\n"
table_str += makeTableEntries(df_v, keys=df_algorithm_keys, objs = textured_objects,
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                              agg_title = 'Textured-DF')
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys, objs = textured_objects,
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                              agg_title = 'Textured-PC')
table_str += "\\hline\n"
table_str += "\\hline\n"
table_str += makeTableEntries(df_v, keys=df_algorithm_keys, objs = textureless_objects,
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                              agg_title = 'Textureless-DF')
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys, objs = textureless_objects,
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                              agg_title = 'Textureless-PC')
table_str += "\\hline\n"
table_str += "\\hline\n"
table_str += makeTableEntries(df_v, keys=df_algorithm_keys, 
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                             agg_title = 'All-DF')
table_str += makeTableEntries(pc_v, keys=pc_algorithm_keys, 
                              val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max, 
                             agg_title = 'All-PC')
#table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
table_str += makeTableEnding(caption = 'AUC of Symmetric Average Distance Error of Distribution Mode')
print(table_str)

\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{5}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Mix & Dropout & Reg Iso & Reg Full \\
\hline
master chef can
 & -2.29 & -0.78 & -2.82 & 1.01 & -1.91 & -0.97 & -0.94 & -1.04 & -1.66 & \textbf{1.21} \\
cracker box
 & -2.29 & \textbf{4.03} & -1.00 & 2.26 & -1.91 & 1.13 & 1.68 & 1.14 & 3.75 & 1.81 \\
sugar box
 & -2.29 & 3.77 & 2.64 & 2.24 & -1.80 & 0.04 & 0.97 & 1.95 & \textbf{5.94} & -0.06 \\
tomato soup can
 & -2.29 & 0.90 & -2.24 & 1.61 & -1.73 & -0.27 & -0.02 & 1.16 & \textbf{2.02} & 1.63 \\
mustard bottle
 & -2.29 & 3.85 & -1.21 & 2.15 & -1.73 & 2.38 & 2.41 & 1.12 & \textbf{4.61} & 1.73 \\
tuna fish can
 & -2.29 & -3.12 & -5.86 & \textbf{0.42} & -2.05 & -1.17 & -2.19 & -0.96 & -0.20 & 0.31 \\
pudding box
 & -2.29 & 2.18 & -0.18 & -1.35 & -1.95 & 1.15 & 1.19 & 0.82 & \textbf{2.64} & 0.47 \\
gelatin box
 & -2.2

In [38]:
all_item_tables = {}
from functools import partial

#for k, v in results.items():
for k in results_keys:
    v = results[k]
    table_str = ''
    table_str += makeTableHeading([h.replace('_', ' ') for h in v.keys()],
                                  heading = table_heading)
    if(k == 'likelihood'):
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = non_sym_objs, 
                                      val_func=mean_log_clean, bold_func = max,
                                      agg_title = 'Non-Symmetric-DF')
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = sym_objs,
                                      val_func=mean_log_clean, bold_func = max, 
                                      agg_title = 'Symmetric-DF')
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = textured_objects,
                                      val_func=mean_log_clean, bold_func = max, 
                                      agg_title = 'Textured-DF')
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = textureless_objects,
                                      val_func=mean_log_clean, bold_func = max, 
                                      agg_title = 'Textureless-DF')
        table_str += "\\hline\n"
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=mean_log_clean, bold_func = max)
        #table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
        table_str += makeTableEnding(caption = 'Mean Log Likelihood of Ground Truth Orientation')


    elif('angular' in k):
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = non_sym_objs, 
                                      val_func=partial(accuracyAUC100, max_theshold=180), bold_func = max,
                                      agg_title = 'Non-Symmetric-DF')
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = sym_objs, 
                                      val_func=partial(accuracyAUC100, max_theshold=180), bold_func = max, 
                                      agg_title = 'Symmetric-DF')
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = textured_objects, 
                                      val_func=partial(accuracyAUC100, max_theshold=180), bold_func = max, 
                                      agg_title = 'Textured-DF')
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = textureless_objects, 
                                      val_func=partial(accuracyAUC100, max_theshold=180), bold_func = max, 
                                      agg_title = 'Textureless-DF')
        #table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min)
        table_str += "\\hline\n"
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=partial(accuracyAUC100, max_theshold=180), bold_func = max)
        #table_str += makeTableEnding(caption = 'AUC ' + k.replace('_', ' '))
        table_str += makeTableEnding(caption = 'AUC of Symmetric Angular Error of Distribution Mode')
    else:
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = non_sym_objs,
                                      val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max,
                                      agg_title = 'Non-Symmetric-DF')
        table_str += makeTableEntries(v, keys=algorithm_keys, objs = sym_objs,
                                      val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max,
                                      agg_title = 'Symmetric-DF')
        table_str += "\\hline\n"
        #table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min)
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=partial(accuracyAUC100, max_theshold=0.1), bold_func = max)
        #table_str += makeTableEnding(caption = 'AUC ' + k.replace('_', ' '))
        if('sym' in k):
            table_str += makeTableEnding(caption = 'AUC of Symmetric Average Distance Error of Distribution Mode')
        else:
            table_str += makeTableEnding(caption = 'AUC of Average Distance Error of Distribution Mode')
    
    #table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
    all_item_tables[k] = table_str
    print(table_str)
    print('\n\n\n')

\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{5}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Mix & Dropout & Reg Iso & Reg Full \\
\hline
Non-Symmetric-DF 
 & -2.29 & 1.56 & -2.43 & 1.26 & -1.90 & 0.44 & 0.66 & 0.70 & \textbf{2.81} & 1.01 \\
Symmetric-DF 
 & -2.29 & -2.71 & -11.44 & -5.93 & -2.23 & -2.34 & -2.27 & -8.09 & -3.89 & \textbf{-1.71} \\
Textured-DF 
 & -2.29 & 1.28 & -2.50 & 1.36 & -1.88 & 0.35 & 0.57 & 0.60 & \textbf{2.80} & 1.01 \\
Textureless-DF 
 & -2.29 & \textbf{-0.23} & -7.56 & -3.23 & -2.13 & -0.96 & -0.82 & -4.22 & -1.13 & -0.58 \\
\hline
All 
 & -2.29 & 0.86 & -3.90 & 0.09 & -1.95 & -0.01 & 0.18 & -0.74 & \textbf{1.71} & 0.57 \\
\hline
\end{tabular}
\caption{Mean Log Likelihood of Ground Truth Orientation}
\end{table*}





\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{

In [151]:
all_item_tables = {}
from object_pose_utils.utils.pose_error import accuracyAUC
def accuracyAUC100(x, max_theshold):
    return accuracyAUC(x, max_theshold)*100

from functools import partial

#for k, v in results.items():
for k in results_keys:
    v = results[k]
    table_str = ''
    table_str += makeTableHeading([h.replace('_', ' ') for h in v.keys()], 
                                  heading = table_heading)
    if(k == 'likelihood'):
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=mean_log_clean, bold_func = max, individual = True)
        #table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
        table_str += makeTableEnding(caption = 'Mean Log Likelihood of Ground Truth Orientation using DenseFusion')
    elif('angular' in k):
        #table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min, individual = True)    
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=partial(accuracyAUC100, max_theshold=180), 
                                      bold_func = max, individual = True)
        #table_str += makeTableEnding(caption = 'AUC ' + k.replace('_', ' '))
        table_str += makeTableEnding(caption = 'AUC of Symmetric Angular Error of Distribution Mode using DenseFusion')
    else:
        #table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min, individual = True)
        table_str += makeTableEntries(v, keys=algorithm_keys, val_func=partial(accuracyAUC100, max_theshold=0.1), 
                                      bold_func = max, individual = True)
        #table_str += makeTableEnding(caption = 'AUC ' + k.replace('_', ' '))
        if('sym' in k):
            table_str += makeTableEnding(caption = 'AUC of Symmetric Average Distance Error of Distribution Mode using DenseFusion')
        else:
            table_str += makeTableEnding(caption = 'AUC of Average Distance Error of Distribution Mode using DenseFusion')
    
    all_item_tables[k] = table_str
    print(table_str)
    print('\n\n\n')


\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{5}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Mix & Dropout & Reg Iso & Reg Full \\
\hline
master chef can
 & -2.29 & -0.78 & -2.82 & 1.01 & -1.91 & -0.97 & -0.94 & -1.04 & -1.66 & \textbf{1.21} \\
cracker box
 & -2.29 & \textbf{4.03} & -1.00 & 2.26 & -1.91 & 1.13 & 1.68 & 1.14 & 3.75 & 1.81 \\
sugar box
 & -2.29 & 3.77 & 2.64 & 2.24 & -1.80 & 0.04 & 0.97 & 1.95 & \textbf{5.94} & -0.06 \\
tomato soup can
 & -2.29 & 0.90 & -2.24 & 1.61 & -1.73 & -0.27 & -0.02 & 1.16 & \textbf{2.02} & 1.63 \\
mustard bottle
 & -2.29 & 3.85 & -1.21 & 2.15 & -1.73 & 2.38 & 2.41 & 1.12 & \textbf{4.61} & 1.73 \\
tuna fish can
 & -2.29 & -3.12 & -5.86 & \textbf{0.42} & -2.05 & -1.17 & -2.19 & -0.96 & -0.20 & 0.31 \\
pudding box
 & -2.29 & 2.18 & -0.18 & -1.35 & -1.95 & 1.15 & 1.19 & 0.82 & \textbf{2.64} & 0.47 \\
gelatin box
 & -2.2

\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c|c|c|c|c|c||}
\hline
  &  & \multicolumn{4}{c|}{Histogram} & \multicolumn{5}{c||}{Bingham} \\
\hline
Objects & Uniform & Conf & Reg & Comp & Cosine & Fixed & Mix & Dropout & Reg Iso & Reg Full \\
\hline
master chef can
 & 85.00 & 93.89 & 92.88 & 93.27 & 93.09 & 94.39 & 94.39 & 40.20 & \textbf{94.41} & \textbf{94.41} \\
cracker box
 & 70.90 & 90.66 & 85.45 & 86.78 & 86.97 & 91.73 & 91.73 & 22.71 & \textbf{91.78} & \textbf{91.78} \\
sugar box
 & 77.29 & 94.08 & 91.50 & 91.35 & 89.80 & 95.34 & 95.34 & 31.61 & \textbf{95.39} & \textbf{95.39} \\
tomato soup can
 & 88.35 & 95.15 & 93.92 & 94.95 & 94.77 & \textbf{95.70} & \textbf{95.70} & 33.12 & 95.69 & 95.69 \\
mustard bottle
 & 79.79 & 95.12 & 92.27 & 92.38 & 92.20 & \textbf{96.30} & \textbf{96.30} & 41.88 & 96.26 & 96.26 \\
tuna fish can
 & 90.83 & 95.60 & 94.54 & 95.34 & 93.73 & \textbf{96.00} & \textbf{96.00} & 44.76 & 95.99 & 95.99 \\
pudding box
 & 87.00 & 92.54 & 88.

In [10]:
old_hist_results = 'ral_results/single_img_results.npz'
old_results = {}
updateResults(old_results, old_hist_results)

In [11]:
print(mean_log_clean(results['likelihood']['hist_conf'][19]))
print(mean_log_clean(results['likelihood']['hist_comp_global'][19]))


-5.550682973753043
-10.450504445126201


In [12]:
np.log(results['likelihood']['hist_comp_global'][19])

array([-2.76398614e+01, -1.17265607e+01, -1.05128298e+01, -9.56166938e+00,
       -1.03226954e+01, -1.30577700e+01, -3.15368584e+01, -2.82804491e+01,
       -2.39361721e+01, -1.15214879e+01, -2.75959299e+01, -2.88752009e+01,
       -2.81989687e+01, -2.73482227e+01, -3.30842368e+01, -2.95222406e+01,
       -1.61589364e+01, -1.68491082e+01, -1.65632775e+01, -2.41959320e+01,
       -1.67768797e+01, -1.29167911e+01, -1.42840955e+01, -2.22151622e+01,
       -1.89400312e+01, -2.05515898e+01, -1.60218617e+01, -1.22245653e+01,
       -1.73861522e+01, -1.50851754e+01, -1.68272508e+01, -1.62041168e+01,
       -3.25336002e+01, -2.06854247e+01, -2.59648622e+01, -2.38920554e+01,
       -1.23620870e+01, -2.13059571e+01, -3.48734111e+01, -3.46345636e+01,
       -1.01829968e+01, -2.57339772e+01, -2.80193338e+01, -2.25879527e+01,
       -3.51070262e+01, -3.36519326e+01, -2.39199952e+01, -4.34715733e+00,
       -4.53029025e+01, -3.78957040e+01, -1.10287235e+01, -1.13765045e+01,
       -1.31321557e+01, -